In [1]:
%load_ext autoreload
%autoreload 2

# The Ch35t client

My client for the Ch35t format is still at an early stage, but it already
implements some basic features including loading/parsing chest files and
dealing with different types of data both for the hint and for the payload.

## Parser

The parser takes care of, well... parsing :-) the JSON files describing chests.
It provides methods to manually parse JSONs from URLs, files, or strings, 
however these methods are mostly used internally as the constructor tries
to automatically detect what it is dealing with.

In [1]:
import parser

# if a Parser is instantiated with no input params, it
# will have an empty json
cp = parser.Parser()
print(cp.json())

None


## Manually load a chest
### From file

In [2]:
cp._load_file("examples/dummy.json")
cp.json()

{'label': {'name': 'I am a dummy chest',
  'URI': 'http://3564030356.org/ch35t/examples/dummy.json',
  'author': 'mala@sdf.org'},
 'hint': {'origin': 'http://this.is.the/hint/origin',
  'data': 'This is the hint data.',
  'format': 'text/plain'},
 'payload': {'data': 'This is simple, unuseful, plain text payload data.',
  'format': 'text/plain'}}

### From URL

In [ ]:
url = "http://3564020356.org/ch35t/examples/dummy.json"
cp._load_url(url)
cp.json()

### From string

In [6]:
jj = '''
{
  "label": {
    "name": "I am a dummy chest",
    "URI": "http://3564030356.org/ch35t/examples/dummy.json",
    "author": "mala@sdf.org"
  },
  "hint": {
    "origin": "http://this.is.the/hint/origin",
    "data": "This is the hint data.",
    "format": "text/plain"
  },
  "payload": {
    "data": "This is simple, unuseful, plain text payload data.",
    "format": "text/plain"
  }
}'''
cp._load_string(jj)
cp.json()

{'label': {'name': 'I am a dummy chest',
  'URI': 'http://3564030356.org/ch35t/examples/dummy.json',
  'author': 'mala@sdf.org'},
 'hint': {'origin': 'http://this.is.the/hint/origin',
  'data': 'This is the hint data.',
  'format': 'text/plain'},
 'payload': {'data': 'This is simple, unuseful, plain text payload data.',
  'format': 'text/plain'}}

*NOTE* that manual load won't probably be required.

## Load automatically
Parser is able to automatically detect whether the JSON chest description is being passed as an URL, a file, or a string. If it is initalised
with a string that starts with `http://` or `https://` it will automatically
call load_url, if it starts with `file://` it will automatically load a file,
otherwise it will load a string.
As we might want a URL to be considered as a string and not as a reference to
some content, it is still possible to do that by instantiating an emtpy parser
and loading the url as a string.

In [7]:
cp = parser.Parser(jj)
cp.json()

{'label': {'name': 'I am a dummy chest',
  'URI': 'http://3564030356.org/ch35t/examples/dummy.json',
  'author': 'mala@sdf.org'},
 'hint': {'origin': 'http://this.is.the/hint/origin',
  'data': 'This is the hint data.',
  'format': 'text/plain'},
 'payload': {'data': 'This is simple, unuseful, plain text payload data.',
  'format': 'text/plain'}}

In [6]:
cp = parser.Parser("http://3564020356.org/ch35t/examples/dummy.json")
cp.json()

{'hint': {'data': 'this is the hint', 'format': 'text/plain'},
 'payload': {'data': 'this is the payload',
  'format': 'text/plain',
  'method': 'No method'}}

In [8]:
cp = parser.Parser("file://examples/dummy.json")
cp.json()

{'label': {'name': 'I am a dummy chest',
  'URI': 'http://3564030356.org/ch35t/examples/dummy.json',
  'author': 'mala@sdf.org'},
 'hint': {'origin': 'http://this.is.the/hint/origin',
  'data': 'This is the hint data.',
  'format': 'text/plain'},
 'payload': {'data': 'This is simple, unuseful, plain text payload data.',
  'format': 'text/plain'}}

# Dump hint and payload contents

Hint and payload can be *printed* and can be *dumped*. If one explicitly calls their *dump* methods, a dump of their content (not the raw json, but not the handled content yet) is returned.

To actually *print* them (via their `__str__` method) we will need to additional set a context for the respective objects. This provides extra info required for proper handling of the payloads (right now, just an `output_dir` field specifying where chest data will be stored).

In [9]:
# dump can be run without setting context
print(cp.hint().dump())
print(cp.payload().dump())

[i] Hint
    Origin: http://this.is.the/hint/origin
    Data: This is the hint data.
    Format: text/plain

[i] Payload
    Origin: None
    Data: This is simple, unuseful, plain text payload data.
    Format: text/plain
    Method: None



# Build hint object

Every hint is characterised by some `data` content and a `format`. Depending on the format, the hint will be interpreted in different ways (see e.g. https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/MIME_types/Common_types):

* `text/plain`: data can be simply printed
* `text/html`: html that needs to be interpreted/shown properly
* `image/*`: data will be displayed by the appropriate application
* `application/zip`: a zip file, to be unpacked in the chest directory
* `application/octet-stream`: this is a generic file (perhaps one wants to make the hint a riddle too?)

NOTE that if we want to have BOTH online and offline stuff, we could have an `origin` for URL origins, `format` for their remote formats, `data` for the actual data that is downloaded from the URL and base64-encoded to allow for offline access.

SO:
* the fields under `hint` are `origin`, `format`, and `data`
* if we want to specify a hint by reference we provide `origin` (any URL) and `format` (the format of the data that will be returned by that URL, if the app has to parse it)
* if we want to specify a hint by value we provide `data` in a way that recalls [data URLs](https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URLs), i.e. as in
```
data:[<mediatype>][;base64],<data>
```
* `mediatype` is specified as `format` and if `origin` and `data` are both present they share the same mediatype. Also format is, by default, `text/plain`
* `base64` is provided at the beginning of `data` if we want the data to be provided as encoded (great if we have binaries)
* `data` is the actual data. Differently from data URLs, it is NOT provided as URI-encoded

In [17]:
import hint as h

jj = '''{
"hint": { 
  "origin": "http://3564020356.org/deserve.htm",
  "data": "base64,TUFMIFRJUlJVRVpGIENSIE1BTCBSS1pZSU9MIEVYIE1BTCBPSVkgVUFFIFJJQ0YgIk1BTCBBQ1dBTFJNIERZRVVQTEZXTCBDUiBNRSBEWUVVIE1BSU0gVUwgSVpMIFJLWlpFS1lGTEYgR0ggT0hSTUxaSCI=" 
}}'''

context = { "output_dir": "/tmp/chests" }
cp._load_string(jj)
print(f"From this json:\n{cp.json()}")
hint = h.Hint(cp.json())
hint.set_ctx(context)
print(f"\n... We get the following hint:\n{hint}")

From this json:
{'hint': {'origin': 'http://3564020356.org/deserve.htm', 'data': 'base64,TUFMIFRJUlJVRVpGIENSIE1BTCBSS1pZSU9MIEVYIE1BTCBPSVkgVUFFIFJJQ0YgIk1BTCBBQ1dBTFJNIERZRVVQTEZXTCBDUiBNRSBEWUVVIE1BSU0gVUwgSVpMIFJLWlpFS1lGTEYgR0ggT0hSTUxaSCI='}}

... We get the following hint:
MAL TIRRUEZF CR MAL RKZYIOL EX MAL OIY UAE RICF "MAL ACWALRM DYEUPLFWL CR ME DYEU MAIM UL IZL RKZZEKYFLF GH OHRMLZH"


# Build payload

Note that at the moment we do not have a `toString` implementation for payloads yet, so printing them will just return the object

In [1]:
import payload as p

jj = '''{
"payload": { 
  "data": "this is the payload"
}
}'''

cp._load_string(jj)
cp.json()
payload = p.Payload(cp.json())
print(payload)

# Playing Ch35t

Rather than providing a client to play with chests, I am trying to build
primitives that can be used by them to do so. All of this is to say, for
now there are only two ways to play:

* through these notebook cells :-)
* using the CLI tool (ie. running `python cli.py` from a terminal)

The `ch35t.Chest` class is initialised with a chest (file, URL, or string)
and then allows you to show the chest's hint and unlock the chest.
Depending on the format of the hint and the payload, different actions
are taken.

## Deserve

The example below loads a very simple chest, which provides the hint as
plain text and has no real contents, just a password (the key) which is
encrypted using MD5 and that you need to guess

In [14]:
from ch35t import Chest

def show_hint(chest):
    print(f"You found a chest named '{chest.name()}'.")
    print(f"It carries a hint for its unlocking:\n{chest.hint}")

c = Chest("file://examples/deserve.json", chests_dir="./chests")
show_hint(c)

You found a chest named 'Deserve Me'.
It carries a hint for its unlocking:
MAL TIRRUEZF CR MAL RKZYIOL EX MAL OIY UAE RICF "MAL ACWALRM DYEUPLFWL CR ME DYEU MAIM UL IZL RKZZEKYFLF GH OHRMLZH"


... Note that if you do not want to immediately show the hint's data in the JSON file you can provide it as base64-encoded, as in the following example

In [15]:
c = Chest("file://examples/deserve_b64.json", chests_dir="./chests")
show_hint(c)

You found a chest named 'Deserve Me - A base64 variant'.
It carries a hint for its unlocking:
MAL TIRRUEZF CR MAL RKZYIOL EX MAL OIY UAE RICF "MAL ACWALRM DYEUPLFWL CR ME DYEU MAIM UL IZL RKZZEKYFLF GH OHRMLZH"


... now it's your turn to provide the key to open the chest!

In [16]:
def unlock_chest(chest):
    key = input("Enter the key to unlock this chest:")
    if (chest.unlock(key)):
        print("Yay, you provided the right key and unlocked the chest!")
    else:
        print("The key is not correct. The chest is still locked.")
        
unlock_chest(c)

Enter the key to unlock this chest:Whatever
The key is not correct. The chest is still locked.


# Riddle 02

The following example has a similar payload, i.e. a MD5-encrypted password
that has to be guessed. However, the hint is a bit more complicated: its data
is provided as a zip file, so the `show_hint` method will take care of
unpacking it and showing you its contents. You will then need to go to the
hint's directory and study the hint (this one is already quite harder than
the previous one!) before you try guessing the key

In [17]:
from ch35t import Chest

c = Chest("file://examples/riddle02.json", chests_dir="./chests")
show_hint(c)

You found a chest named 'riddle02'.
It carries a hint for its unlocking:
Files are available in ./chests:
riddle02/
riddle02/+Ma's Reversing Riddle 02_files/
riddle02/+Ma's Reversing Riddle 02_files/0c.zip
riddle02/+Ma's Reversing Riddle 02_files/alice.gif
riddle02/+Ma's Reversing Riddle 02_files/skull02.jpg
riddle02/+Ma's Reversing Riddle 02.html



In [20]:
unlock_chest(c)

Enter the key to unlock this chest:Whatever
The key is not correct. The chest is still locked.


# Hic Sunt Leones

This section is unexplored territory! Here I am just playing with different
libraries to test new functionalities (I'll remove it if this bothers you
but I thought it would have been a nice peek into what I am playing with...)

## Chest signatures

Testing which libs to use to enable chest signing

In [21]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)
public_key = private_key.public_key()
pad = padding.PSS(
    mgf=padding.MGF1(hashes.SHA256()),
    salt_length=padding.PSS.MAX_LENGTH
)

In [22]:
message = b"A message I want to sign"
bad_message = b"A message different from the one I signed"

signature = private_key.sign(
    message,
    pad,
    hashes.SHA256()
)

In [23]:
for m in [message, bad_message]:
    try:
        public_key.verify(
            signature,
            m,
            pad,
            hashes.SHA256()
        )
        print(f"{m}: good messaage")
    except Exception as e:
        print(f"{m}: bad message")

b'A message I want to sign': good messaage
b'A message different from the one I signed': bad message


## Playing with schemas

In the following cells I am playing with `jsonschema` to automatically
validate my example json files. I then extend the check so it runs as
a unit test (will provide more broken jsons in `tests/bad_json` in the 
future). This should ideally allow anyone to work on extensions of the
schema without breaking the client wrt existing JSON files

In [24]:
# import fastjsonschema
from jsonschema import validate, ValidationError
import json

In [25]:
with open("schema/1.0.0.json", "rt") as f:
    s = json.load(f)
# print(schema)

with open("examples/deserve_b64.json", "rt") as f:
    riddle = json.load(f)

try:
    validate(schema=s, instance=riddle)
    print("Validation successful")
except ValidationError as e:
    print(e.message)

Validation successful


In [26]:
import unittest
from tests.validation import TestSchema
unittest.main(argv=[''], verbosity=2, exit=False)

test_bad_jsons (tests.validation.TestSchema) ... ok
test_good_jsons (tests.validation.TestSchema) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.055s

OK


## Automatically generate schema documentation

aaand here's how the `schema.md` file is generated :-)

In [27]:
import json
import jsonschema2md

parser = jsonschema2md.Parser(
    examples_as_yaml=False,
    show_examples="all",
)
with open("./schema/1.0.0.json", "r") as json_file:
    md_lines = parser.parse_schema(json.load(json_file))
print(''.join(md_lines))

with open ("schema.md", "w") as f:
    f.write(''.join(md_lines))



# Ch35t Schema

*Chests as defined in the Ch35t format*

## Properties

- **`label`** *(object)*: A label attached to the chest, containing information about it. Cannot contain additional properties.
  - **`name`** *(string, required)*: The chest name.
  - **`URI`** *(string, format: uri)*: A Uniform Resource Identifier for the chest (can match the URL it can be downloaded from.
  - **`author`** *(string, format: email)*: The email address of the chest creator (often used in conjuction with a signature.
- **`hint`** *(object)*: A hint that might help you find the key to open the chest. Cannot contain additional properties.
  - **`origin`** *(string, format: uri)*: A Uniform Resource Identifier for the hint (can match the URL it can be downloaded from).
  - **`data`** *(string)*: The hint contents (plain text or base64-encoded binary).
  - **`format`**: The format (mime-type) of the hint contents. Note that we currently allow only formats for which we have a handler, but this is not str